## Working directory set-up

In [1]:
%%capture
_drive = False
_local = True
try: 
  from google.colab import drive
  drive.mount('/content/drive')
  _drive = True
  _local = False
except: 
  print('no google drive')

In [2]:
cd drive/MyDrive/PhD/Thesis/Dissertation/Model

[WinError 3] The system cannot find the path specified: 'drive/MyDrive/PhD/Thesis/Dissertation/Model'
C:\Users\amal\Desktop\Model


In [3]:
import sys
sys.path
#sys.path.remove('C:\\Program Files\\NVIDIA Corporation\\NVSMI')

['C:\\Users\\amal\\Desktop\\Model',
 'C:\\ProgramData\\anaconda3\\python310.zip',
 'C:\\ProgramData\\anaconda3\\DLLs',
 'C:\\ProgramData\\anaconda3\\lib',
 'C:\\ProgramData\\anaconda3',
 '',
 'C:\\Users\\amal\\AppData\\Roaming\\Python\\Python310\\site-packages',
 'C:\\ProgramData\\anaconda3\\lib\\site-packages',
 'C:\\ProgramData\\anaconda3\\lib\\site-packages\\win32',
 'C:\\ProgramData\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\ProgramData\\anaconda3\\lib\\site-packages\\Pythonwin']

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 68.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [5]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)

if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  print(tf.test.gpu_device_name())

Tue May  2 20:23:50 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090       WDDM | 00000000:01:00.0  On |                  Off |
|  0%   39C    P8               25W / 450W|    899MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization, LSTM, Conv1D
from keras.models import Sequential
from dataclasses import dataclass
import tensorflow_datasets as tfds 
import sklearn
from sklearn.model_selection import train_test_split
import pickle

from keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping

# HP tuning 
from tensorboard.plugins.hparams import api as hp
import optuna

import pandas as pd
import numpy as np
from statistics import mean

from datetime import datetime
import time
import glob
import re
from pprint import pprint

# visualization 
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [8]:
print("tf version:",tf.__version__)
print("keras version:", keras.__version__)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)
print("sklearn version:", sklearn.__version__)

tf version: 2.10.0
keras version: 2.10.0
pandas version: 1.5.0
numpy version: 1.22.4
sklearn version: 1.2.2


In [9]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_22:08:44_Pacific_Standard_Time_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


## Set-up Hyperparameters

In [10]:

@dataclass
class Config:
  
    # preprocessing params
    MAX_LEN = 256       #512 in bert
    VOCAB_SIZE = 30000  # 30000 in bert     

    # mask  15% in bert
    KW_MASK_RATE = 0.25
    RAND_MASK_RATE = 0.20

    # model params
    EMBED_DIM = 512#128
    FF_DIM = 2024#128  # number of units 768 in bert
    NUM_LAYERS = 12#3 # 3, 6, 12 in bert

    LR = 0.001   # 0.0001 in bert
    DROPOUT = 0.1 # 0.1 in bert
    BATCH_SIZE = 64#32     #256 in bert

    # transformer 
    NUM_HEAD = 16 #8 # 8,16 in atten, 12 in bert

    # parallelism
    BUFFER_SIZE = tf.data.experimental.AUTOTUNE

    CODE_VERSION = 'Not defined'

    def save(self, fname):
      dict1 = {'MAX_LEN' : self.MAX_LEN, 
               'VOCAB_SIZE' : self.VOCAB_SIZE,
               'EMBED_DIM' : self.EMBED_DIM,
               'BATCH_SIZE' : self.BATCH_SIZE,
               'KW_MASK_RATE' : self.KW_MASK_RATE,
               'RAND_MASK_RATE' : self.RAND_MASK_RATE,
               'LR' : self.LR,
               'DROPOUT' : self.DROPOUT,
               'FF_DIM' : self.FF_DIM,
               'NUM_LAYERS' : self.NUM_LAYERS,
               'NUM_HEAD' : self.NUM_HEAD,
               'CODE_VERSION' : self.CODE_VERSION,
               
               }
      file1 = open(fname, "w") 
      str1 = repr(dict1)
      file1.write("config = " + str1 + "\n")
      file1.close()

config = Config()

## Load the data

In [11]:
def get_data(file):

  opened=False
  try:
    f = open(file,'r', encoding='utf-8')
    data = " ".join(line.strip() for line in f) 
    opened = True
  except Exception:
    pass
  

  if opened:
    return data
  else:
    print('+++ Couldnt open the file: ', file)
    return ''

In [12]:
def load_data_from_file(fpath):
    
    xmi_pathlist = pd.read_csv (fpath)

    dataset = pd.DataFrame()

    dataset['xmi']= xmi_pathlist['File'].apply(get_data)
    dataset['label'] = 1

    return dataset

In [13]:
def get_keywords(fpath):
  keywords_fname = 'keywords.txt'
  keywords = [line.rstrip() for line in open(keywords_fname)]
  return keywords

## Preprocessing

### Filtering

In [14]:
def data_filter(data):
  fdata = pd.DataFrame([x for a, x in data.iterrows() if len(x[0].split(' ')) > 50]).reset_index(drop =True)
  return fdata

### Truncation

In [15]:
def truncateText(text ,max_len):

    cursor = 0  
    end = max_len
    lst = []
    text = text.split(' ')

    while (end < len(text)):
        substr = text[cursor : end]
        fulltxt = ' '.join(substr)
        fulltxt = fulltxt.encode(encoding="ascii",errors="ignore").decode()
        lst.append(fulltxt)
        cursor =  end        
        end = (cursor+max_len) if (cursor+max_len<len(text)) else len(text)        

    if (end-cursor) > (max_len/2): 
        fulltxt = ' '.join(text[cursor : end])
        fulltxt = fulltxt.encode(encoding="ascii",errors="ignore").decode()
        lst.append(fulltxt)

    return lst


In [16]:
def truncation(data):
  fdata = pd.DataFrame()
  listdict = []

  for a, text in data.iterrows():
    t_text = truncateText(text[0], config.MAX_LEN) 
    listdict.append(pd.DataFrame(data = {'xmi':t_text, 'label': text[1]}))
  
  fdata = pd.concat(list(listdict), axis=0, ignore_index =True) 

  return fdata

### Tokenization

In [17]:
def get_vectorize_layer(texts, vocab_size, max_seq, vec_log='vec_layer_logs/tv_layer.pkl', special_tokens=["[mask]"]):

    try: 
      from_disk = pickle.load(open(vec_log, "rb"))
      vectorize_layer = TextVectorization.from_config(from_disk['config'])
      vectorize_layer.set_weights(from_disk['weights'])
      print('vectorize layer already existed')

    except:
      print('vectorize layer not found')

      vectorize_layer = TextVectorization(
          max_tokens=vocab_size,
          output_mode="int",
          standardize=custom_standardization,
          output_sequence_length=max_seq,
      )
      text_ds = tf.data.Dataset.from_tensor_slices(texts).prefetch(config.BUFFER_SIZE)
      vectorize_layer.adapt(text_ds)

      # Insert mask token in vocabulary
      vocab = vectorize_layer.get_vocabulary()
      vocab = list(dict.fromkeys(vocab))

      vocab = vocab[2 : vocab_size - len(special_tokens)] + special_tokens
      vectorize_layer.set_vocabulary(vocab)

      # Pickle the config and weights
      pickle.dump({'config': vectorize_layer.get_config(),
              'weights': vectorize_layer.get_weights()}
              , open(vec_log, "wb"))

    return vectorize_layer

In [18]:
def custom_standardization(input_data):
    
    new_data = tf.strings.regex_replace(input_data, "=", " = ")
    new_data = tf.strings.regex_replace(new_data, r'[^a-zA-Z\[\]<>:\/=\s]', "")
    new_data = tf.strings.regex_replace(new_data, r'<', ' <')
    new_data = tf.strings.regex_replace(new_data, r'>', '> ')

    return new_data

In [19]:
def encode(texts):
    texts = tf.expand_dims(texts, -1)  
    encoded_texts = tf.squeeze(vectorize_layer(texts))
    return encoded_texts

def decode(tokens):
    temp = []
    for t in tokens:
        if t != 0 and t < len(id2token):
            temp.append(id2token[t])
    return " ".join(temp)

## Proxytask

In [20]:
def get_keywords_pos(text, keywords_dic):
  keywors_pos_list = [] 

  for i in range(text.shape[0]):
    temp = []
    # get the position of keywords
    for a in range(0, len(keywords)):
      temp.append(np.flatnonzero(text[i] == keywords_dic[keywords[a]]))

    #combine multiple np.arrays to one, return array of np.arrays 
    keywors_pos_list.append(np.concatenate(temp))

  return keywors_pos_list

In [21]:
def get_masked_input_and_labels(encoded_txt):
    
    encoded_texts = []
    for row in encoded_txt:
        encoded_texts.append(row.tolist())

    encoded_texts = np.array(encoded_texts, int)

    #texts_masked = np.copy(encoded_texts)
    texts_masked =  np.copy(encoded_texts)
    label_list =  np.copy(encoded_texts)

    #Num of instances
    _m = encoded_texts.shape[0]

    # Get keywords posotions list 
    keywors_pos_list = get_keywords_pos(encoded_texts, keywords_dic)
    #print(keywors_pos_list, MASK_TOKEN_ID)

    inp_mask = []
    for i in range(np.shape(keywors_pos_list)[0]):
      # Create random mask on keywords
      rand = (np.random.rand(np.shape(keywors_pos_list[i])[0]) < config.KW_MASK_RATE)
      inp_mask.append(rand) 


    selection = []
    for i in range(_m):
        selection.append(keywors_pos_list[i][((inp_mask[i] == True))])      

    # Apply the maske by replacing the selected tokens with MASK_TOKEN_ID
    for i in range(_m):
      texts_masked[i, selection[i]] = MASK_TOKEN_ID


    # Randomly mask 10% of the text
    inp_mask_2 =  (np.random.rand(*encoded_texts.shape) <  config.RAND_MASK_RATE)
    # Do not mask special tokens 
    inp_mask_2[encoded_texts <= 2] = False
    # Apply the mask
    texts_masked[inp_mask_2] = MASK_TOKEN_ID 
    
    # Make keywords mask to true
    inp_mask_2[texts_masked == MASK_TOKEN_ID] = True
    # Set targets to -1 by default, it means ignore
    labels = -1 * np.ones(encoded_texts.shape, dtype=int)
    # Set labels for masked tokens
    labels[inp_mask_2] = texts_masked[inp_mask_2]

    texts_masked = np.array(texts_masked, int)
    labels = np.array(labels, int)

    # Prepare sample_weights to pass to .fit() method
    sample_weights = np.ones(labels.shape)
    sample_weights[labels == -1] = 0

    # for sw in sample_weights:
    # #print('+++++++', sw, )
    # #a = np.count_nonzero(sample_weights, axis=1)
    #   if np.count_nonzero(sw) ==0 or np.count_nonzero(sw)==256:
    #       print('+++++++',sw)


    return texts_masked, label_list, sample_weights
    

In [22]:
#x, y, z = get_masked_input_and_labels(np.array([[4448,0,3,100,55], [0,5,294,5,200]]))

In [23]:
#id2token[4]

In [24]:
#token2id['.']

In [25]:
#import tensorflow.python.keras.backend as K
#sess = K.get_session()

In [26]:
#import gc 
#gc.collect()

## Prepare datasets 

### Fine-tuning DS

In [27]:
def get_finetuning_dataset(train_df, test_df):
# Prepare datasets for training

  train_text_ds_raw = tf.data.Dataset.from_tensor_slices(
              tf.cast(train_df.xmi.values, tf.string)
  ) 
  x_train = train_text_ds_raw.map(encode, 
                                    num_parallel_calls=config.BUFFER_SIZE)
  y_train = tf.data.Dataset.from_tensor_slices(
              tf.cast(train_df.label.values, tf.int64),) 

  train_ds = tf.data.Dataset.zip(
      (x_train, y_train )).shuffle(1000).batch(config.BATCH_SIZE).prefetch(config.BUFFER_SIZE)


  # for testing
  test_text_ds_raw = tf.data.Dataset.from_tensor_slices(
              tf.cast(test_df.xmi.values, tf.string)
  ) 
  x_test = test_text_ds_raw.map(encode, 
                                    num_parallel_calls=config.BUFFER_SIZE)
  y_test = tf.data.Dataset.from_tensor_slices(
              tf.cast(test_df.label.values, tf.int64),) 

  test_ds = tf.data.Dataset.zip(
      (x_test, y_test )).shuffle(1000).batch(config.BATCH_SIZE).prefetch(config.BUFFER_SIZE) 

  return train_ds, test_ds

In [28]:
#for each in y_test.take(3):
#  print(each)

In [29]:
#x_train.cardinality().numpy()

In [30]:
#x_test.cardinality().numpy()

### Pre-training DS

In [31]:
def get_pretraining_dataset(X_train, X_valid, X_test): # training %

  trainig_dataset_path = 'Preprocessed datasets/pre-training/training/'
  validation_dataset_path = 'Preprocessed datasets/pre-training/validation/'
  testing_dataset_path = 'Preprocessed datasets/pre-training/testing/'

  try:
    mlm_ds = tf.data.experimental.load(trainig_dataset_path)
    valid_ds = tf.data.experimental.load(validation_dataset_path)
    test_ds = tf.data.experimental.load(testing_dataset_path)
    print('preprocessed datasets already existed')

  except:
    print('preprocessed datasets not found')

    # Prepare data for pre-trained model 
    x_all_raw = tf.data.Dataset.from_tensor_slices(
                tf.cast(X_train.xmi.values, tf.string)) 

    x_all_code = x_all_raw.map(encode, 
                  num_parallel_calls=config.BUFFER_SIZE)

    x_valid_raw = tf.data.Dataset.from_tensor_slices(
                tf.cast(X_valid.xmi.values, tf.string)) 

    x_valid_code = x_valid_raw.map(encode, 
                  num_parallel_calls=config.BUFFER_SIZE)
    
    x_test_raw = tf.data.Dataset.from_tensor_slices(
                tf.cast(X_test.xmi.values, tf.string)) 

    x_test_code = x_test_raw.map(encode, 
                  num_parallel_calls=config.BUFFER_SIZE)
    
    # Applying the Proxytask on training ds
    a = tfds.as_numpy(x_all_code)
    x_masked_train, y_masked_labels, sample_weights = get_masked_input_and_labels(a)

    mlm_ds = tf.data.Dataset.from_tensor_slices(
      ( x_masked_train, y_masked_labels, sample_weights)
      )
    # mlm_ds = tf.data.Dataset.from_tensor_slices(
    #       (
    #       {
    #           "encoder_inputs": x_masked_train,
    #           "decoder_inputs": y_masked_labels,
    #       },
    #       y_masked_labels, sample_weights
    #   )
    #     )
    mlm_ds = mlm_ds.shuffle(1000).batch(config.BATCH_SIZE).prefetch(config.BUFFER_SIZE)

    # Applying the Proxytask on validation and testing datasets
    x_masked_val, y_masked_labels_val, sample_weights_val = get_masked_input_and_labels(tfds.as_numpy(x_valid_code))

    valid_ds = tf.data.Dataset.from_tensor_slices(
      (x_masked_val, y_masked_labels_val, sample_weights_val)
      )
    # valid_ds = tf.data.Dataset.from_tensor_slices(
    #       (
    #       {
    #           "encoder_inputs": x_masked_train,
    #           "decoder_inputs": y_masked_labels,
    #       },
    #       y_masked_labels, sample_weights
    #   )
    #     )
    valid_ds = valid_ds.shuffle(1000).batch(config.BATCH_SIZE).prefetch(config.BUFFER_SIZE)
    
    x_masked_test, y_masked_labels_test, sample_weights_test = get_masked_input_and_labels(tfds.as_numpy(x_test_code))

    test_ds = tf.data.Dataset.from_tensor_slices(
      (x_masked_test, y_masked_labels_test, sample_weights_test)
      )
    
    test_ds = test_ds.shuffle(1000).batch(config.BATCH_SIZE).prefetch(config.BUFFER_SIZE)
    
    # Save datasets
    tf.data.experimental.save(mlm_ds, trainig_dataset_path)
    tf.data.experimental.save(valid_ds, validation_dataset_path)
    tf.data.experimental.save(test_ds, testing_dataset_path)


  return mlm_ds, valid_ds, test_ds, #x_masked_val, y_masked_labels_val

In [32]:
#x_valid_code.cardinality().numpy()

In [33]:
#for each in x_all_code.take(3):
#  print(each)

In [34]:
#len(x_masked_train)

In [35]:
#encode('> < = - +')

In [36]:
#vectorize_layer.get_vocabulary()

## Create DL model 


In [37]:
class TextGenerator(keras.callbacks.Callback):
    def __init__(self, sample_tokens, top_k=5):
        self.sample_tokens = sample_tokens
        self.k = top_k

    def decode(self, tokens):
        temp = []
        for t in tokens:
           if t != 0 and t < len(id2token):
             temp.append(id2token[t])
        return " ".join(temp)

    def convert_ids_to_tokens(self, id):
        if id >= len(id2token):
          return ''
        else:
          return id2token[id]

    def on_epoch_end(self, epoch, logs=None):
        prediction = self.model.predict(self.sample_tokens)

        masked_index = np.where(self.sample_tokens == MASK_TOKEN_ID)
        masked_index = masked_index[1]
        mask_prediction = prediction[0][masked_index]

        top_indices = mask_prediction[0].argsort()[-self.k :][::-1]
        values = mask_prediction[0][top_indices]

        for i in range(len(top_indices)):
            p = top_indices[i]
            v = values[i]
            tokens = np.copy(sample_tokens[0])
            tokens[masked_index[0]] = p
            #print(tokens)
            result = {
                "input_text": self.decode(sample_tokens[0].numpy()),
                "prediction": self.decode(tokens),
                "probability": v,
                "predicted mask token": self.convert_ids_to_tokens(p),
            }
            pprint(result)

In [38]:
class DLModel(keras.Model):
    def train_step(self, inputs):
        if len(inputs) == 3:
            features, labels, sample_weight = inputs

        else:
            features, labels = inputs
            sample_weight = None
        
        #sample_weight = [1]

        with tf.GradientTape() as tape:
            predictions = self(features, training=True)
            loss = loss_fn(labels, predictions, sample_weight=sample_weight)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Compute our own metrics
        loss_tracker.update_state(loss, sample_weight=sample_weight)

        # Return a dict mapping metric names to current value
        return {"loss": loss_tracker.result()}

    @property
    def metrics(self):
        return [loss_tracker]

In [39]:
def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
            if pos != 0
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return pos_enc

### Simple DL model

In [40]:
def create_dl_model():
    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)
    #x_in = layers.Dense(256)(inputs) 
    embeddings = layers.Embedding(config.VOCAB_SIZE, config.EMBED_DIM, name="embedding" )(inputs)
    
    L4 = embeddings

    for i in range(config.NUM_LAYERS):
      L1 = layers.Dense(config.FF_DIM, activation="relu")(L4) 
      #L2 = layers.Dense(config.FF_DIM)(L1)
      L3 = layers.Dropout(rate = config.DROPOUT, )(L1) 
      #L4 = layers.LayerNormalization(epsilon=1e-6, name="layernormalization_"+ str(i))(L3) 

    mlm_output = layers.Dense(config.VOCAB_SIZE, name="mlm_cls", activation="softmax")(L3)

    #mlm_model = DLModel(inputs, mlm_output, name="dl_model")
    mlm_model = keras.Model(inputs, mlm_output, name="dl_model")  

    return mlm_model

### CNN model

In [41]:
def create_cnn_model():

    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)
    embeddings = layers.Embedding(config.VOCAB_SIZE, config.EMBED_DIM, name="embedding" )(inputs)

    L2 = embeddings

    for i in range(config.NUM_LAYERS):
      L1 = Conv1D(filters = config.FF_DIM, kernel_size =3, padding = 'SAME')(L2)
      L2 = layers.ReLU()(L1) 
    #L3 = layers.MaxPool1D( padding = 'SAME')(L2)

    #L4 = layers.Flatten()(L3)
    mlm_output = layers.Dense(config.VOCAB_SIZE, name="mlm_cls", activation="softmax")(L2)

    mlm_model = keras.Model(inputs, mlm_output, name="cnn_model")

    #optimizer = keras.optimizers.Adam(learning_rate=config.LR)
    #mlm_model.compile(optimizer=optimizer)

    return mlm_model

### LSTM model

In [42]:
def create_lstm_model():
    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)
    #x_in = layers.Dense(256)(inputs) 
    embeddings = layers.Embedding(config.VOCAB_SIZE, config.EMBED_DIM, name="embedding" )(inputs)

    L2 = embeddings

    for i in range(config.NUM_LAYERS):
      L1 = LSTM(config.FF_DIM, return_sequences= True)(L2)

      L2 = layers.Dropout(rate = config.DROPOUT, )(L1) 

    mlm_output = layers.Dense(config.VOCAB_SIZE, name="mlm_cls", activation="softmax")(L2)

    mlm_model = keras.Model(inputs, mlm_output, name="lstm_model")

    #optimizer = keras.optimizers.Adam(learning_rate=config.LR)
    #mlm_model.compile(optimizer=optimizer)
    
    return mlm_model

### Transformer model

In [43]:
def transformer_model():

    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)
    # inputs = {
    #   "encoder_inputs": keras.Input((config.MAX_LEN,), dtype=tf.int64),
    #   "decoder_inputs": keras.Input((config.MAX_LEN,), dtype=tf.float32), }

    word_embeddings = layers.Embedding(config.VOCAB_SIZE, config.EMBED_DIM, name="word_embedding")(inputs)
    
    position_embeddings = layers.Embedding(
        input_dim=config.MAX_LEN,
        output_dim=config.EMBED_DIM,
        weights=[get_pos_encoding_matrix(config.MAX_LEN, config.EMBED_DIM)],
        name="position_embedding",)(tf.range(start=0, limit=config.MAX_LEN, delta=1))
    
    embeddings = word_embeddings + position_embeddings

    encoder_output = embeddings

    for i in range(config.NUM_LAYERS):
      encoder_output = transformer_Encoder(encoder_output, encoder_output, encoder_output, i)
      decoder_output = transformer_Decoder(embeddings, embeddings, embeddings, encoder_output, i)

    mlm_output = layers.Dense(config.VOCAB_SIZE, name="mlm_cls", activation="softmax")(decoder_output)
    
    mlm_model = keras.Model(inputs, mlm_output, name="transformer_model")

    #optimizer = keras.optimizers.Adam(learning_rate=config.LR)

    #mlm_model.compile(optimizer=optimizer)

    return mlm_model

In [44]:
def transformer_Encoder (query, key, value, i):

    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(num_heads=config.NUM_HEAD, key_dim=config.EMBED_DIM // config.NUM_HEAD,)(query, key, value)
    
    attention_output = layers.Dropout(config.DROPOUT)(attention_output)
    
    attention_output = layers.LayerNormalization(epsilon=1e-6,)(query + attention_output)

    # Feed-forward layer
    ffn = keras.Sequential(
        [
            layers.Dense(config.FF_DIM, activation="relu"),
            layers.Dense(config.EMBED_DIM),
        ], name="encoder_{}/ffn".format(i),)
    
    ffn_output = ffn(attention_output)
    
    ffn_output = layers.Dropout(config.DROPOUT)(ffn_output)
    
    sequence_output = layers.LayerNormalization(epsilon=1e-6)(attention_output + ffn_output)
    
    return sequence_output

def transformer_Decoder (query, key, value, enc_output, i):

    # Multi headed self-attention 1
    attention_output = layers.MultiHeadAttention(num_heads=config.NUM_HEAD, key_dim=config.EMBED_DIM // config.NUM_HEAD,)(query, key, value)
    
    attention_output = layers.Dropout(config.DROPOUT)(attention_output)
    
    attention_output = layers.LayerNormalization(epsilon=1e-6,)(query + attention_output)

    # Multi headed self-attention 2
    attention_output2 = layers.MultiHeadAttention(num_heads=config.NUM_HEAD, key_dim=config.EMBED_DIM // config.NUM_HEAD,)(attention_output, enc_output, enc_output)
    
    attention_output2 = layers.Dropout(config.DROPOUT)(attention_output2)
    
    attention_output2 = layers.LayerNormalization(epsilon=1e-6,)(attention_output + attention_output2)

    # Feed-forward layer
    ffn = keras.Sequential(
        [
            layers.Dense(config.FF_DIM, activation="relu"),
            layers.Dense(config.EMBED_DIM),
        ], name="decoder_{}/ffn".format(i),)
    
    ffn_output = ffn(attention_output2)
    
    ffn_output = layers.Dropout(config.DROPOUT)(ffn_output)
    
    sequence_output = layers.LayerNormalization(epsilon=1e-6)(attention_output2 + ffn_output)
    
    return sequence_output


## Pre-training

In [45]:
def pre_train_model(train_ds, valid_ds, epochs, model_name, callbacks, config, logs):
  tf.get_logger().setLevel('ERROR')
  strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

  with strategy.scope():
    model = get_model(model_name)

    # optimizer =_optimizer #keras.optimizers.Adam(learning_rate=config.LR)
    # loss =  keras.losses.SparseCategoricalCrossentropy()
    # #loss_tracker = keras.metrics.Mean(name="loss_mean") #tf.keras.metrics.Accuracy() #t

    # model.compile(
    #     optimizer=optimizer,
    #     loss='sparse_categorical_crossentropy',
    #     weighted_metrics=["sparse_categorical_accuracy"], #jit_compile=True,
    # )
  # model.summary()
  
  # craet the batch 
  #train_ds = train_ds.shuffle(1000).batch(config.BATCH_SIZE).prefetch(config.BUFFER_SIZE)
  #valid_ds = valid_ds.shuffle(1000).batch(config.BATCH_SIZE).prefetch(config.BUFFER_SIZE)

  History = model.fit(train_ds, epochs=epochs,  validation_data = valid_ds, callbacks=callbacks)

  eval_results = model.evaluate(valid_ds, verbose ='0', batch_size=config.BATCH_SIZE)
  loss = eval_results[0] #mean(History.history['val_loss']) #'val_sparse_categorical_accuracy' to get accuracy on validation daraset
  accuracy = eval_results[1] #mean(History.history['val_sparse_categorical_accuracy']) #'val_sparse_categorical_accuracy' to get accuracy on validation daraset
  
  model.save(logs + "/" + model.name + ".h5")
  #model.save("dl_model.h5", include_optimizer =False)
  config.save(logs + "/" + model.name + "_config.txt")

  return accuracy, loss, model

In [46]:
def get_model(model_name):
  # tf.get_logger().setLevel('ERROR')
  # strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

  # with strategy.scope():

  if model_name == 'DL': model = create_dl_model()
  if model_name == 'LSTM': model = create_lstm_model()
  if model_name == 'CNN': model = create_cnn_model()
  if model_name == 'Transformer': model = transformer_model()

  optimizer = keras.optimizers.Adam(learning_rate=config.LR)
  loss =  keras.losses.SparseCategoricalCrossentropy()
  #loss_tracker = keras.metrics.Mean(name="loss_mean") #tf.keras.metrics.Accuracy() #t

  model.compile(
        optimizer= optimizer,
        loss='sparse_categorical_crossentropy',
        weighted_metrics=["sparse_categorical_accuracy"], #jit_compile=True,
  )
  #model.summary()

  return model

In [47]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

  def get_config(self):
    LR_config = {
    'd_model': self.d_model,
    'warmup_steps': self.warmup_steps,

     }
    return LR_config

## ----- Main

In [48]:
def pre_processing(data, _filtering = True, _truncation= True):
  new_data = data[0:][['xmi', 'label']]
  
  if _drive: 
    new_data = data[0:150][['xmi', 'label']]
  
  new_data['xmi'] = new_data['xmi'].astype(str)

  if _filtering:
    new_data = data_filter(new_data)

  if _truncation: 
    new_data = truncation(new_data)

  return new_data

In [49]:
def get_callbacks(sample_tokens, log_dir):
  #sample_tokens = vectorize_layer(["[mask] processJarAttrs ( ) throws BuildException" ])
  #sample_tokens = vectorize_layer(["[mask] index = 1" ])
  generator_callback = TextGenerator(sample_tokens.numpy())
  
  tboard_callback = keras.callbacks.TensorBoard(log_dir = log_dir,
                                                  histogram_freq = 1,
                                                  profile_batch = '20,31')
  
  earlystopping = EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True, verbose=1)
  #Hparam_callback = hp.KerasCallback(logdir, hparams)

  return [generator_callback, tboard_callback, earlystopping]

In [50]:
def split_data(data, training_ratio= 0.8, _valid=False):

  train_size=0.8

  X = data.drop(columns = ['label']).copy()
  y = data['label']

  X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=training_ratio)

  if  _valid == False:
    return X_train, X_rem, y_train, y_rem
  
  else:
    X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
    return  X_train, X_valid, X_test, y_train, y_valid, y_test


In [51]:
def get_clean_dataste (ds):
    clean_ds =pd.DataFrame()

    for index, row in ds.iterrows():
        version_name= ''
        encoding_name= ''

        content =row['xmi'] 
        version = re.search(r'version\s*=\s*["|\']([0-9].[0-9])["|\']', content)# content.decode('utf-8'))
        encoding = re.search(r'encoding\s*=\s*["|\']([a-zA-Z0-8-]*)["|\']', content)# content.decode('utf-8'))

        content_clean = re.sub(r'[^a-zA-Z<>:\/=\s]', '', content)
        _isascii = content_clean.isascii()
        content_clean = re.sub(r'=', ' = ', content_clean)
        content_clean = re.sub(r'<', ' <', content_clean)
        content_clean = re.sub(r'>', '> ', content_clean)

        if version:
            version_name = version.group(1)

        if encoding:
            encoding_name = encoding.group(1)

        df2 =  pd.DataFrame({'xmi': content_clean, 
                                            'version': version_name,
                                            'encoding': encoding_name,
                                            'isascii': _isascii,
                                            'label':1}, index=[0])

        if version_name != '' and _isascii:
            clean_ds = pd.concat([clean_ds, df2])
        

    return clean_ds

In [52]:
def get_existing_DS_VEC():
    
    vec_layer_logs = 'vec_layer_logs/'+''+'tv_layer.pkl'
    vectorize_layer = get_vectorize_layer(
    [],#X_train.xmi.values.tolist(),
    config.VOCAB_SIZE,
    config.MAX_LEN, 
    vec_log= vec_layer_logs,
    special_tokens= ['[mask]'],
    )
    
    mlm_train_ds, mlm_valid_ds, mlm_test_ds= get_pretraining_dataset(X_train=None, X_valid=None, X_test=None)
    
    
    return vec_layer_logs, mlm_train_ds, mlm_valid_ds, mlm_test_ds


In [73]:
dataset = load_data_from_file('Datasets/Pre-training/Pre-training_datasets.csv')

keywords = get_keywords('keywords.txt')

+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.results4care.PerinatologieOverdracht-v0.1_9.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_catrip_tumor_registry_v1_rv1_17.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_catissue-cacore-sdk_catissuesuitev11_1.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_DCM-Under-Development_nl.results4care.PerinatologieConsult-v0.1_14.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.parelsnoer.Lichaamstemperatuur.v0.91_46.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.results4care.Visus-v0.5_148.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.nfu.OverdrachtGewicht-v1.1_90.xmi
+++ Couldnt open the file:  

+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_DCM-Under-Development_nl.results4care.PAMmini-v0.1_17.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_caaers-docs_Annotated_caAERS_v1_rv9_10.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_cadsr-objectcart_ObjectCart1.0_1.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_org.hl7.GlasgowComaScale(15pointversion)-v0.75_115.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.nfu.OverdrachtLengte-v1.1_142.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_DCM-Under-Development_nl.OD4H.KwaliteitvanLevenmini-v0.4_12.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_cacore-sdk_sdk_2.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_common-biorepository-model_CBM with

+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.nfu.OverdrachtLevensovertuiging-v1.1_109.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_cabio_caBIO_4.3_1.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_lexevs-remote_sdk_2.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.parelsnoer.Woonsituatie-v0.1_87.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_DCM-Under-Development_nl.results4care.PeriferePulsaties-v0.2_13.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_DCM-Under-Development_nl.results4care.BayleyScalesInfantDevelopment-BSID-v0.7_5.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_common-biorepository-model_CBMnolocal_2.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_calims_sdk_1.xmi
+++ 

+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.PRN.Alcoholgebruik-v0.4_52.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.nfu.OverdrachtTekstUitslag-v1.1_71.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.parelsnoer.Arbeidssituatie-v0.3_123.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.PRN.FoetaleGroeivertraging-v0.3_91.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.results4care.pijnscoreVASNRS-v0.991_118.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.nfu.OverdrachtGlasgowComaScale-v1.1_114.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_caaers-docs_caAERS-caDSR_

+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.nfu.OverdrachtDrugsgebruik-v1.1_37.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_catissue-dynamic-extensions_caTissueSuite_10.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.nfu.OverdrachtTemperatuur-v1.1_80.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_catissue-core_org_specimen_model_10.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_catissue-core_caTissueCore_v2.0_15.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_DCM-Under-Development_nl.results4care.PHQmini-v0.1_16.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/NCIP_catissue-core_Query_Model_1.xmi
+++ Couldnt open the file:  Datasets/Pre-training/Lindholmen/DetailedClinicalModels_Detailed-Clinical-Models_nl.results4care.Ademhali

In [74]:
dataset = get_clean_dataste(dataset)
#print(pd.unique(dataset['version']))
#print(pd.unique(dataset['encoding']))

In [75]:
all_data = pre_processing(dataset[['xmi', 'label']], _truncation= True)

In [76]:
#X_train, X_valid, X_test ,_,_,_ = split_data(all_data, training_ratio= 0.8, _valid=True)
X_train, X_test , _,_ = split_data(all_data, training_ratio= 0.8, _valid=False)

In [52]:
# Get vec layer for tokenization
vec_layer_logs = 'vec_layer_logs/'+''+'tv_layer.pkl'
keywords = get_keywords('keywords.txt')
vectorize_layer = get_vectorize_layer(keywords+
    [],#X_train.xmi.values.tolist(),
    config.VOCAB_SIZE,
    config.MAX_LEN, 
    vec_log= vec_layer_logs,
    special_tokens= ['[mask]'],
)

vectorize layer already existed


In [53]:
# Get mask token id for masked language model
MASK_TOKEN_ID = vectorize_layer(["[mask]"]).numpy()[0][0]
print('--------------', MASK_TOKEN_ID)

# Get dictionary of keywords
keywords_dic = {keyword: encode(keyword).numpy().tolist()[0] for keyword in keywords} #keywords_dic[keywords[1]]
#keywords_dic = {decode( vectorize_layer([keyword])[0].numpy()): encode(keyword).numpy().tolist()[0] for keyword in keywords} #keywords_dic[keywords[1]]

# Get dictionary of tokens ids
id2token = dict(enumerate(vectorize_layer.get_vocabulary()))  #id2token[1]
token2id = {y: x for x, y in id2token.items()}  #token2id['[UNK]']

-------------- 29999


In [54]:
#train_ds, test_ds =  get_finetuning_dataset(train_df, test_df)
#mlm_train_ds, mlm_valid_ds, mlm_test_ds= get_pretraining_dataset(X_train, X_valid, X_test)
mlm_train_ds, mlm_valid_ds, mlm_test_ds= get_pretraining_dataset(X_train=None, X_valid=None, X_test=None)
#mlm_train_ds, mlm_valid_ds, mlm_test_ds= get_pretraining_dataset(X_train, X_test, X_test)

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.
preprocessed datasets already existed


In [ ]:
mls =sa

In [64]:
config.CODE_VERSION = 'UML_DL_v4'
MODEL_NAME = 'Transformer' # DL, LSTM, CNN, Transformer
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")+ '_' + MODEL_NAME

sample_tokens = vectorize_layer(['[mask] isSpecification="false"  isLeaf="false" isRoot="false" xmi.id="m1" isAbstract="false" name="Model UML" >' ])
callbacks= get_callbacks(sample_tokens, logs) 

model = get_model(MODEL_NAME)
_,_, model= pre_train_model(mlm_train_ds, mlm_test_ds, 5, MODEL_NAME, callbacks, config, logs)

Epoch 1/5
   73/47887 [..............................] - ETA: 1:51:51 - loss: 0.7108 - sparse_categorical_accuracy: 0.4535

KeyboardInterrupt: 

In [61]:
%load_ext tensorboard
%tensorboard --logdir logs

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [62]:
#python C:\users\amal\appdata\roaming\python\python310\site-packages\tensorboard\main.py --logdir=C:\Users\amal\Desktop\Model\logs

## Make predictions

In [ ]:
def decode(tokens):
    temp = []
    for t in tokens:
        if t != 0 and t < len(id2token):
            temp.append(id2token[t])
    return " ".join(temp)

def convert_ids_to_tokens(id):
    if id >= len(id2token):
        return ''
    else:
        return id2token[id]

def get_predictions(prediction, k):
  masked_index = np.where(test_tokens == MASK_TOKEN_ID)
  masked_index = masked_index[1]
  mask_prediction = prediction[0][masked_index]

  top_indices = mask_prediction[0].argsort()[-k:][::-1]
  values = mask_prediction[0][top_indices]

  for i in range(len(top_indices)):
      p = top_indices[i]
      v = values[i]
      tokens = np.copy(test_tokens[0])
      tokens[masked_index[0]] = p
              #print(tokens)
      result = {
          "input_text": decode(test_tokens[0].numpy()),
          "prediction": decode(tokens),
          "probability": v,
          "predicted mask token": convert_ids_to_tokens(p),
      }
      pprint(result)

# test_tokens = vectorize_layer(["[mask] text = ' some text '" ])
# prediction = model.predict(test_tokens)
# top_k =3

# get_predictions(prediction, top_k)

In [ ]:
#model_path =logs+ "/transformer_model.h5"
#loaded_model = keras.models.load_model(model_path,  custom_objects={"DLModel": DLModel})

#test_tokens = vectorize_layer(["[mask] text = ' some text '" ])
#prediction = loaded_model.predict(test_tokens)
#top_k =3

#get_predictions(prediction, top_k)

In [ ]:
#np.allclose(model.predict(test_tokens), loaded_model.predict(test_tokens))

## Hyperparameter Tuning 

In [53]:
vec_layer_logs, mlm_train_ds, mlm_valid_ds, mlm_test_ds = get_existing_DS_VEC()

vectorize layer already existed
Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.
preprocessed datasets already existed


In [54]:
# Clear any logs from previous runs
#!rm -rf ./logs/hparam_tuning

In [73]:
def check_existed_params(df, params): 

  if MODEL_NAME == 'Transformer':
      existed_params = df.loc[(df['state'] == 'COMPLETE') & (df['params_Learning rate'] == params['Learning rate']) & 
                          (df['params_Dropout rate'] == params['Dropout rate'])& 
                          (df['params_Num. layers'] == params['Num. layers'])& 
                          (df['params_Layers Dim'] == params['Layers Dim'])& 
                          (df['params_Emmbedding Dim.'] == params['Emmbedding Dim.'])& 
                          (df['params_Num. heads'] == params['Num. heads'])] 
  else:
      existed_params = df.loc[(df['value'] != 100) & (df['state'] == 'COMPLETE') &  (df['params_Learning rate'] == params['Learning rate']) & 
                          (df['params_Dropout rate'] == params['Dropout rate'])& 
                          (df['params_Num. layers'] == params['Num. layers'])& 
                          (df['params_Layers Dim'] == params['Layers Dim'])& 
                          (df['params_Emmbedding Dim.'] == params['Emmbedding Dim.'])
                          ] 
  
  print('====== existed_params ', existed_params)
  
  return not existed_params.empty

In [68]:
def run(run_dir, hparams):
    
    log_dir =  run_dir.replace('hparam_tuning/','')
    tboard_callback = keras.callbacks.TensorBoard(log_dir = log_dir,
                                                  histogram_freq = 1,
                                                  profile_batch = '20,31')
    
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial

        start = time.time()
        accuracy, loss, _ = pre_train_model(mlm_train_ds, mlm_valid_ds, 5, MODEL_NAME, tboard_callback, config, run_dir)
        training_time = round((time.time() - start)/60)

        tf.summary.scalar('Accuracy', accuracy, step=1)
        tf.summary.scalar('Loss', loss, step=1)
        tf.summary.scalar('Time (Min.)', training_time, step=1)

        return accuracy, loss

In [69]:
def objective(trial):

    #model_name = trial.suggest_categorical('model', ['DL', 'CNN', 'LSTM', 'Transformer'])
    lr = trial.suggest_categorical('Learning rate', [0.0001, 0.001, 0.01])
    dropout_rate = trial.suggest_categorical('Dropout rate', [0.1, 0.2, 0.3])
    #batch_size = trial.suggest_categorical('Batch size', [ 64,128,256])
    num_layers = trial.suggest_categorical('Num. layers', [3, 6, 12])
    num_units = trial.suggest_categorical('Layers Dim', [512, 1024, 2048])
    emb_dim = trial.suggest_categorical('Emmbedding Dim.', [128, 256, 512])
    
    
    hparams = {
                  #'model': model_name,
                  'Learning rate': lr,
                  'Dropout rate': dropout_rate,
                  #'Batch size': batch_size,
                  'Num. layers': num_layers,
                  'Layers Dim': num_units,
                  'Emmbedding Dim.': emb_dim,

              }
    print(hparams)

    #config.BATCH_SIZE = batch_size
    config.NUM_LAYERS = num_layers
    config.FF_DIM = num_units
    config.EMBED_DIM = emb_dim
    config.LR = lr

    if MODEL_NAME == 'Transformer':
      num_heads = trial.suggest_categorical('Num. heads', [8, 16])
      hparams['Num. heads'] = num_heads
      config.NUM_HEAD = num_heads

        
    df = study.trials_dataframe()
    try:
      if check_existed_params(df, hparams): #not existed_params.empty:
        print('================== param already existed')
        return 100 #study.trials_dataframe().value[a['number']][1]
    except:
      pass

    run_name = "run-%d" % trial.number
    accuracy, loss = run(hp_logs + '/' + run_name, hparams)
    trial.set_user_attr("accuracy", accuracy)


    return loss

In [75]:
config.CODE_VERSION = 'DL_model_v4'
MODEL_NAME = 'LSTM' # DL, LSTM, CNN, Transformer
hp_logs = 'logs/hparam_tuning/' + 'LSTM 25-04-2023' #MODEL_NAME+ datetime.now().strftime(" %d-%m-%Y")
optuna_logs = "logs/optuna/" + 'LSTM 25-04-2023' #MODEL_NAME+ datetime.now().strftime(" %d-%m-%Y")

study_name = MODEL_NAME # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)


study = optuna.create_study(study_name= MODEL_NAME, 
                            direction='minimize', 
                            storage =storage_name, 
                            pruner=optuna.pruners.MedianPruner(),
                            load_if_exists =True) #maximize val accuracy or min val loss

study.optimize(objective, n_trials=25)

pickle.dump(study, open(optuna_logs +".pkl", "wb"))
study.trials_dataframe().to_csv(optuna_logs + '.csv')


trial = study.best_trial

print('Loss: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2023-05-10 12:43:49,380] Using an existing study with name 'LSTM' instead of creating a new one.


{'Learning rate': 0.001, 'Dropout rate': 0.3, 'Num. layers': 6, 'Layers Dim': 1024, 'Emmbedding Dim.': 512}
====== existed_params  Empty DataFrame
Columns: [number, value, datetime_start, datetime_complete, duration, params_Dropout rate, params_Emmbedding Dim., params_Layers Dim, params_Learning rate, params_Num. layers, user_attrs_accuracy, state]
Index: []
Epoch 1/5
47887/47887 [==============================] - 12431s 260ms/step - loss: 0.1048 - sparse_categorical_accuracy: 0.8950 - val_loss: 0.0598 - val_sparse_categorical_accuracy: 0.9327
Epoch 2/5
47887/47887 [==============================] - 12463s 260ms/step - loss: 0.0585 - sparse_categorical_accuracy: 0.9340 - val_loss: 0.0555 - val_sparse_categorical_accuracy: 0.9370
Epoch 3/5
47887/47887 [==============================] - 12461s 260ms/step - loss: 0.0551 - sparse_categorical_accuracy: 0.9371 - val_loss: 0.0532 - val_sparse_categorical_accuracy: 0.9392
Epoch 4/5
47887/47887 [==============================] - 12446s 260ms/st

[I 2023-05-11 06:24:59,803] Trial 85 finished with value: 0.05100821703672409 and parameters: {'Learning rate': 0.001, 'Dropout rate': 0.3, 'Num. layers': 6, 'Layers Dim': 1024, 'Emmbedding Dim.': 512}. Best is trial 7 with value: 0.04913010448217392.
[I 2023-05-11 06:24:59,911] Trial 86 finished with value: 100.0 and parameters: {'Learning rate': 0.001, 'Dropout rate': 0.3, 'Num. layers': 6, 'Layers Dim': 1024, 'Emmbedding Dim.': 512}. Best is trial 7 with value: 0.04913010448217392.
[I 2023-05-11 06:25:00,024] Trial 87 finished with value: 100.0 and parameters: {'Learning rate': 0.001, 'Dropout rate': 0.3, 'Num. layers': 6, 'Layers Dim': 1024, 'Emmbedding Dim.': 512}. Best is trial 7 with value: 0.04913010448217392.


{'Learning rate': 0.001, 'Dropout rate': 0.3, 'Num. layers': 6, 'Layers Dim': 1024, 'Emmbedding Dim.': 512}
====== existed_params      number     value             datetime_start          datetime_complete  \
85      85  0.051008 2023-05-10 12:43:49.404200 2023-05-11 06:24:59.787690   

                 duration  params_Dropout rate  params_Emmbedding Dim.  \
85 0 days 17:41:10.383490                  0.3                     512   

    params_Layers Dim  params_Learning rate  params_Num. layers  \
85               1024                 0.001                   6   

    user_attrs_accuracy     state  
85             0.941434  COMPLETE  
================== param already existed
{'Learning rate': 0.001, 'Dropout rate': 0.3, 'Num. layers': 6, 'Layers Dim': 1024, 'Emmbedding Dim.': 512}
====== existed_params      number     value             datetime_start          datetime_complete  \
85      85  0.051008 2023-05-10 12:43:49.404200 2023-05-11 06:24:59.787690   

                 duration  

[I 2023-05-11 06:25:00,150] Trial 88 finished with value: 100.0 and parameters: {'Learning rate': 0.001, 'Dropout rate': 0.3, 'Num. layers': 6, 'Layers Dim': 1024, 'Emmbedding Dim.': 512}. Best is trial 7 with value: 0.04913010448217392.
[I 2023-05-11 06:25:00,257] Trial 89 finished with value: 100.0 and parameters: {'Learning rate': 0.001, 'Dropout rate': 0.3, 'Num. layers': 6, 'Layers Dim': 1024, 'Emmbedding Dim.': 512}. Best is trial 7 with value: 0.04913010448217392.


...

{'Learning rate': 0.001, 'Dropout rate': 0.2, 'Num. layers': 6, 'Layers Dim': 2048, 'Emmbedding Dim.': 512}
====== existed_params      number     value             datetime_start          datetime_complete  \
95      95  0.051408 2023-05-12 00:07:59.745274 2023-05-13 11:10:45.237881   

                 duration  params_Dropout rate  params_Emmbedding Dim.  \
95 1 days 11:02:45.492607                  0.2                     512   

    params_Layers Dim  params_Learning rate  params_Num. layers  \
95               2048                 0.001                   6   

    user_attrs_accuracy     state  
95              0.94128  COMPLETE  
================== param already existed
{'Learning rate': 0.001, 'Dropout rate': 0.2, 'Num. layers': 6, 'Layers Dim': 2048, 'Emmbedding Dim.': 512}
====== existed_params      number     value             datetime_start          datetime_complete  \
95      95  0.051408 2023-05-12 00:07:59.745274 2023-05-13 11:10:45.237881   

                 duration  

[I 2023-05-13 11:10:45,784] Trial 100 finished with value: 100.0 and parameters: {'Learning rate': 0.001, 'Dropout rate': 0.2, 'Num. layers': 6, 'Layers Dim': 2048, 'Emmbedding Dim.': 512}. Best is trial 7 with value: 0.04913010448217392.
[I 2023-05-13 11:10:45,884] Trial 101 finished with value: 100.0 and parameters: {'Learning rate': 0.001, 'Dropout rate': 0.2, 'Num. layers': 6, 'Layers Dim': 2048, 'Emmbedding Dim.': 512}. Best is trial 7 with value: 0.04913010448217392.


{'Learning rate': 0.001, 'Dropout rate': 0.2, 'Num. layers': 6, 'Layers Dim': 2048, 'Emmbedding Dim.': 512}
====== existed_params      number     value             datetime_start          datetime_complete  \
95      95  0.051408 2023-05-12 00:07:59.745274 2023-05-13 11:10:45.237881   

                 duration  params_Dropout rate  params_Emmbedding Dim.  \
95 1 days 11:02:45.492607                  0.2                     512   

    params_Layers Dim  params_Learning rate  params_Num. layers  \
95               2048                 0.001                   6   

    user_attrs_accuracy     state  
95              0.94128  COMPLETE  
================== param already existed
{'Learning rate': 0.01, 'Dropout rate': 0.3, 'Num. layers': 6, 'Layers Dim': 1024, 'Emmbedding Dim.': 512}
====== existed_params  Empty DataFrame
Columns: [number, value, datetime_start, datetime_complete, duration, params_Dropout rate, params_Emmbedding Dim., params_Layers Dim, params_Learning rate, params_Num. l

[W 2023-05-13 12:24:13,237] Trial 102 failed with parameters: {'Learning rate': 0.01, 'Dropout rate': 0.3, 'Num. layers': 6, 'Layers Dim': 1024, 'Emmbedding Dim.': 512} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\amal\AppData\Roaming\Python\Python310\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\amal\AppData\Local\Temp\ipykernel_270508\2670375988.py", line 45, in objective
    accuracy, loss = run(hp_logs + '/' + run_name, hparams)
  File "C:\Users\amal\AppData\Local\Temp\ipykernel_270508\2591222207.py", line 12, in run
    accuracy, loss, _ = pre_train_model(mlm_train_ds, mlm_valid_ds, 5, MODEL_NAME, tboard_callback, config, run_dir)
  File "C:\Users\amal\AppData\Local\Temp\ipykernel_270508\1424400836.py", line 23, in pre_train_model
    History = model.fit(train_ds, epochs=epochs,  validation_data = valid_ds, callbacks=callbacks)
  File "C:\Users\amal\Ap

KeyboardInterrupt: 

# Test saved model and trial

In [76]:
pickle.dump(study, open(optuna_logs +".pkl", "wb"))
saved_study = pickle.load(open(optuna_logs + ".pkl", "rb"))
saved_trial = saved_study.best_trial
saved_study.trials_dataframe().to_csv(optuna_logs + '.csv')

print('Accuracy: {}'.format(saved_trial.value))
print("Best hyperparameters: {}".format(saved_trial.params))

Accuracy: 0.04913010448217392
Best hyperparameters: {'Dropout rate': 0.3, 'Emmbedding Dim.': 256, 'Layers Dim': 2048, 'Learning rate': 0.001, 'Num. layers': 3}


In [ ]:
model_path = "logs/hparam_tuning/CNN 28-10-2022/run-24/cnn_model.h5"
loaded_model = keras.models.load_model(model_path)

optimizer = keras.optimizers.Adam()

loaded_model.compile(
        optimizer= optimizer,
        loss='sparse_categorical_crossentropy',
        weighted_metrics=["sparse_categorical_accuracy"], #jit_compile=True,
  )
    
eval_results = loaded_model.evaluate(mlm_valid_ds, batch_size=config.BATCH_SIZE)
loss = eval_results[0]
accuracy = eval_results[1] 

print(loss, accuracy)

In [ ]:
plot_parallel_coordinate(saved_study)
plot_parallel_coordinate(saved_study, params=["Batch size", "Dropout rate"])

In [ ]:
plot_param_importances(study =saved_study, 
                       #evaluator=optuna.importance.MeanDecreaseImpurityImportanceEvaluator()
                       )

In [ ]:
plot_param_importances(study =study, 
                       #evaluator=optuna.importance.MeanDecreaseImpurityImportanceEvaluator()
                       )

In [ ]:
optuna.importance.FanovaImportanceEvaluator()

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_slice(study)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/hparam_tuning

In [ ]:
#%tensorboard --inspect --logdir logs/hparam_tuning

In [ ]:
!pip install optuna-dashboard

In [ ]:
!optuna-dashboard sqlite:///db.sqlite3

## Fine-tuning 

In [ ]:
'''
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

print(data.shape)

print(data)
'''
# Load pretrained model
dl_model = keras.models.load_model("dl_model.h5", custom_objects={"DLModel": DLModel})

pretrained_model = tf.keras.Model(dl_model.input, dl_model.get_layer("dense_179").output)

# Freeze it
pretrained_model.trainable = False
#pretrained_model.layers[3].trainable = False

def create_classifier_model():
    inputs = layers.Input((config.MAX_LEN,), dtype=tf.int64)
    sequence_output = pretrained_model(inputs)
    pooled_output = layers.GlobalMaxPooling1D()(sequence_output)
    outputs = layers.Dense(1, activation="sigmoid")(pooled_output)
    classifer_model = keras.Model(inputs, outputs, name="classification")
    optimizer = keras.optimizers.Adam()
    classifer_model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    return classifer_model


classifer_model = create_classifier_model()
classifer_model.summary()

# Train the classifier with frozen  stage
classifer_model.fit(train_ds, epochs=5, validation_data=test_ds,)

# Unfreeze the model for fine-tuning
pretrained_model.trainable = True
optimizer = keras.optimizers.Adam()

classifer_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
classifer_model.fit(train_ds, epochs=5, validation_data=test_ds, )

In [ ]:
#test_ds.get_single_element()

## Evaluation


In [ ]:
preds = classifer_model.predict(vectorize_layer(['void addConfiguredService ( Service service ) { if ( service . getMessage ( ) == null ) throw new BuildException ( "\"<service>\" must have a \"message\" attribute" ) ; String menuItem = service . getMenuItem ( ) ; if ( menuItem == null ) throw new BuildException ( "\"<service>\" must have a \"menuItem\" attribute" ) ; if ( ! menuItems . add ( menuItem ) ) throw new BuildException ( "\"<service>\" \"menuItem\" value must be unique" ) ; if ( service . getSendTypes ( ) . isEmpty ( ) && service . getReturnTypes ( ) . isEmpty ( ) ) throw new BuildException ( "\"<service>\" must have either a \"sendTypes\" attribute, a \"returnTypes\" attribute or both" ) ; String keyEquivalent = service . getKeyEquivalent ( ) ; if ( ( keyEquivalent != null ) && ( 1 != keyEquivalent . length ( ) ) ) throw new BuildException ( "\"<service>\" \"keyEquivalent\" must be one character if present" ) ; String timeoutString = service . getTimeout ( ) ; if ( timeoutString != null ) { long timeout = - 1 ; try { timeout = Long . parseLong ( timeoutString ) ; } catch ( NumberFormatException nfe ) { throw new BuildException ( "\"<service>\" \"timeout\" must be a positive integral number" ) ; } if ( timeout < 0 ) throw new BuildException ( "\"<service>\" \"timeout\" must not be negative" ) ; } bundleProperties . addService ( service ) ; }',
                                         "void copyApplicationStub ( ) throws BuildException { File newStubFile = new File ( mMacOsDir , bundleProperties . getCFBundleExecutable ( ) ) ; if ( mVerbose ) log ( 'Copying Java application stub to \'' + bundlePath ( newStubFile ) + '\'' ) ; try { mFileUtils . copyFile ( mStubFile , newStubFile ) ; } catch ( IOException ex ) { throw new BuildException ( 'Cannot copy Java Application Stub: ' + ex ) ; } setExecutable ( newStubFile ) ; }",
                                         'void processJavaFileSets ( ) { processCopyingFileSets ( mJavaFileSets , mJavaDir , false ) ; }']))

In [ ]:
preds

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices({'a': ([[2,2,2,2], [2,2,2,2]]),
                                              'b': [5, 6]})

In [ ]:
dataset.element_spec['a']

In [ ]:
dataset.element_spec['b']

In [ ]:
mlm_ds

In [ ]:
valid_ds

In [ ]:
def truncateText(texts ,max_len):
     cursor = 0  
     end = max_len
     lst = []
     text = texts.split(' ')
     while (end < len(text)):
         substr = text[cursor : end]
         lst.append(substr)
         cursor =  end        
         end = (cursor+max_len) if (cursor+max_len<len(text)) else len(text)
         print('cursor', cursor, 'end', end)            
         

     if (end-cursor) > (max_len/2): 
       lst.append(text[cursor : end])


     return lst


txt = "This paper proposes a method that allows non parallel many to many voice conversion by using a variant of a generative adversarial network called StarGAN."

cvv = truncateText(txt,4)
for cv in cvv:
    print(len(cv), cv)

In [ ]:
# Load the dataset
X, Y = load_dataset()

# Create model for KerasClassifier
def create_model(hparams1=dvalue,
                 hparams2=dvalue,
                 ...
                 hparamsn=dvalue):
    # Model definition
    ...

model = KerasClassifier(build_fn=create_model) 

# Specify parameters and distributions to sample from
hparams1 = randint(1, 100)
hparams2 = ['elu', 'relu', ...]
...
hparamsn = uniform(0, 1)

# Prepare the Dict for the Search
param_dist = dict(hparams1=hparams1, 
                  hparams2=hparams2, 
                  ...
                  hparamsn=hparamsn)

# Search in action!
n_iter_search = 16 # Number of parameter settings that are sampled.
random_search = RandomizedSearchCV(estimator=model, 
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   n_jobs=, 
								   cv=, 
								   verbose=)
random_search.fit(X, Y)

# Show the results
print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))